In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier as ABC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split as tts

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
sofifa = pd.read_csv('sofifa.csv')
FBall = pd.read_csv('FutBolMatches.csv')

In [3]:
# Begin pre-processing
FBall['draw'] = FBall.strong_score == FBall.weak_score
FBall['win'] = FBall.strong_score > FBall.weak_score
FBall['lose'] = FBall.strong_score < FBall.weak_score
FBall = FBall.sort_values(by=['strong_team', 'weak_team','win'])
FBall.columns

Index(['Unnamed: 0', 'date', 'strong_team', 'weak_team', 'strong_score',
       'weak_score', 'tournament', 'strong_overall', 'weak_overall',
       'strong_attack', 'weak_attack', 'strong_midfield', 'weak_midfield',
       'strong_deffense', 'weak_deffense', 'strong_bu_speed', 'weak_bu_speed',
       'strong_bu_passing', 'weak_bu_passing', 'strong_bu_positioning',
       'weak_bu_positioning', 'strong_cc_crossing', 'weak_cc_crossing',
       'strong_cc_passing', 'weak_cc_passing', 'strong_cc_shooting',
       'weak_cc_shooting', 'strong_cc_positioning', 'weak_cc_positioning',
       'strong_aggression', 'weak_aggression', 'strong_preassure',
       'weak_preassure', 'strong_avg_age', 'weak_avg_age', 'win',
       'dif_overall', 'dif_attack', 'dif_midfield', 'dif_deffense',
       'dif_bu_speed', 'dif_bu_passing', 'dif_cc_crossing', 'dif_cc_passing',
       'dif_cc_shooting', 'dif_aggression', 'dif_preassure', 'dif_avg_age',
       'draw', 'lose'],
      dtype='object')

In [4]:
sofifa_1_most_rec = sofifa[sofifa.datestats == '5/17/2018']
sofifa_1_most_rec = sofifa_1_most_rec.add_suffix('_sofifa1')
sofifa_2_most_rec = sofifa[sofifa.datestats == '4/30/2018']
sofifa_2_most_rec = sofifa_2_most_rec.add_suffix('_sofifa2')
sofifa_3_most_rec = sofifa[sofifa.datestats == '3/29/2018']
sofifa_3_most_rec = sofifa_3_most_rec.add_suffix('_sofifa3')

In [5]:
FBall = pd.merge(FBall, sofifa_1_most_rec, left_on='strong_team', right_on='countrystats_sofifa1')
FBall.columns

Index(['Unnamed: 0', 'date', 'strong_team', 'weak_team', 'strong_score',
       'weak_score', 'tournament', 'strong_overall', 'weak_overall',
       'strong_attack', 'weak_attack', 'strong_midfield', 'weak_midfield',
       'strong_deffense', 'weak_deffense', 'strong_bu_speed', 'weak_bu_speed',
       'strong_bu_passing', 'weak_bu_passing', 'strong_bu_positioning',
       'weak_bu_positioning', 'strong_cc_crossing', 'weak_cc_crossing',
       'strong_cc_passing', 'weak_cc_passing', 'strong_cc_shooting',
       'weak_cc_shooting', 'strong_cc_positioning', 'weak_cc_positioning',
       'strong_aggression', 'weak_aggression', 'strong_preassure',
       'weak_preassure', 'strong_avg_age', 'weak_avg_age', 'win',
       'dif_overall', 'dif_attack', 'dif_midfield', 'dif_deffense',
       'dif_bu_speed', 'dif_bu_passing', 'dif_cc_crossing', 'dif_cc_passing',
       'dif_cc_shooting', 'dif_aggression', 'dif_preassure', 'dif_avg_age',
       'draw', 'lose', 'datestats_sofifa1', 'countrystats_sof

In [6]:
FBall = pd.concat([FBall,pd.get_dummies(FBall['strong_team'], prefix='strong')],axis=1)
FBall.drop(['strong_team'],axis=1, inplace=True)
FBall = pd.concat([FBall,pd.get_dummies(FBall['weak_team'], prefix='weak')],axis=1)
FBall.drop(['weak_team'],axis=1, inplace=True)
for i in FBall.columns:
    print(i)

Unnamed: 0
date
strong_score
weak_score
tournament
strong_overall
weak_overall
strong_attack
weak_attack
strong_midfield
weak_midfield
strong_deffense
weak_deffense
strong_bu_speed
weak_bu_speed
strong_bu_passing
weak_bu_passing
strong_bu_positioning
weak_bu_positioning
strong_cc_crossing
weak_cc_crossing
strong_cc_passing
weak_cc_passing
strong_cc_shooting
weak_cc_shooting
strong_cc_positioning
weak_cc_positioning
strong_aggression
weak_aggression
strong_preassure
weak_preassure
strong_avg_age
weak_avg_age
win
dif_overall
dif_attack
dif_midfield
dif_deffense
dif_bu_speed
dif_bu_passing
dif_cc_crossing
dif_cc_passing
dif_cc_shooting
dif_aggression
dif_preassure
dif_avg_age
draw
lose
datestats_sofifa1
countrystats_sofifa1
overall_sofifa1
attack_sofifa1
midfield_sofifa1
deffense_sofifa1
bu_speed_sofifa1
bu_passing_sofifa1
bu_positioning_sofifa1
cc_crossing_sofifa1
cc_passing_sofifa1
cc_shooting_sofifa1
cc_positioning_sofifa1
aggression_sofifa1
preassure_sofifa1
avg_age_sofifa1
strong_Arg

In [7]:
desc = FBall.describe()
desc.to_csv('desc.csv')
desc

,Unnamed: 0,strong_score,weak_score,strong_overall,weak_overall,strong_attack,weak_attack,strong_midfield,weak_midfield,strong_deffense,...,weak_South Africa,weak_Spain,weak_Sweden,weak_Switzerland,weak_Turkey,weak_USA,weak_Ukraine,weak_Uruguay,weak_Venezuela,weak_Wales
count,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,...,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000
mean,600.139286,1.606250,0.954464,79.153571,73.991071,79.343750,73.929464,77.938393,72.864286,77.182143,...,0.012500,0.000893,0.027679,0.017857,0.009821,0.037500,0.003571,0.025000,0.023214,0.011607
std,341.436397,1.320545,0.995377,4.026896,4.121253,4.430846,4.868001,4.417120,4.439583,4.332767,...,0.111152,0.029881,0.164123,0.132491,0.098659,0.190068,0.059681,0.156195,0.150651,0.107157
min,1.000000,0.000000,0.000000,68.000000,62.000000,65.000000,60.000000,64.000000,55.000000,65.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,307.750000,1.000000,0.000000,76.000000,71.000000,76.000000,71.000000,74.000000,70.000000,74.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,599.500000,1.000000,1.000000,80.000000,74.000000,80.000000,73.000000,79.000000,73.000000,77.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,896.250000,2.000000,2.000000,83.000000,76.000000,83.000000,77.000000,81.000000,75.000000,81.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1183.000000,8.000000,5.000000,87.000000,85.000000,88.000000,88.000000,86.000000,86.000000,87.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### And now the actual predictive model;

In [26]:
win_classifier = SVC(tol=1e-8, verbose=3, cache_size=500, kernel='linear')
draw_classifier = SVC(tol=1e-8, verbose=3, cache_size=500, kernel='linear')
lose_classifier = SVC(tol=1e-8, verbose=3, cache_size=500, kernel='linear')

In [27]:
y_win = FBall.win
y_draw = FBall.draw
y_lose = FBall.lose

countries = ['strong_Argentina','strong_Australia','strong_Austria','strong_Belgium',
            'strong_Brazil','strong_Bulgaria','strong_Cameroon','strong_Canada',
            'strong_Chile','strong_Colombia','strong_Czech Republic','strong_Denmark',
            'strong_Ecuador','strong_Egypt','strong_England','strong_Finland','strong_France',
            'strong_Germany','strong_Greece','strong_Hungary','strong_Italy','strong_Mexico',
            'strong_Netherlands','strong_Northern Ireland','strong_Norway','strong_Paraguay',
            'strong_Peru','strong_Poland','strong_Portugal','strong_Romania','strong_Russia',
            'strong_Saudi Arabia','strong_Scotland','strong_Slovenia','strong_South Africa',
            'strong_Spain','strong_Sweden','strong_Switzerland','strong_Turkey','strong_Uruguay',
            'strong_Venezuela','strong_Wales','weak_Argentina','weak_Australia','weak_Austria',
            'weak_Belgium','weak_Bolivia','weak_Brazil','weak_Bulgaria','weak_Cameroon',
            'weak_Canada','weak_Chile','weak_China','weak_Colombia','weak_Croatia',
            'weak_Czech Republic','weak_Denmark','weak_Ecuador','weak_Egypt','weak_England',
            'weak_Finland','weak_France','weak_Germany','weak_Greece','weak_Hungary',
            'weak_Iceland','weak_Ireland','weak_Italy','weak_Ivory Coast','weak_Korea Republic',
            'weak_Mexico','weak_Netherlands','weak_New Zealand','weak_Northern Ireland',
            'weak_Norway','weak_Paraguay','weak_Peru','weak_Poland','weak_Portugal',
            'weak_Romania','weak_Russia','weak_Saudi Arabia','weak_Scotland','weak_Slovenia',
            'weak_South Africa','weak_Spain','weak_Sweden','weak_Switzerland','weak_Turkey',
            'weak_USA','weak_Ukraine','weak_Uruguay','weak_Venezuela','weak_Wales']

features = ['strong_overall', 'weak_overall', 'strong_attack', 'weak_attack',
            'strong_midfield', 'weak_midfield', 'strong_deffense', 'weak_deffense',
            'strong_bu_speed', 'weak_bu_speed', 'strong_bu_passing', 'weak_bu_passing', 
            'strong_cc_crossing', 'weak_cc_crossing', 'dif_overall', 'dif_attack', 
            'dif_midfield', 'dif_deffense', 'dif_bu_speed', 'dif_bu_passing',
            'dif_cc_crossing', 'dif_cc_passing', 'dif_cc_shooting', 'dif_aggression', 
            'dif_preassure', 'dif_avg_age', 'overall_sofifa1', 'attack_sofifa1', 
            'midfield_sofifa1','deffense_sofifa1', 'bu_speed_sofifa1', 'bu_passing_sofifa1']

features_new = ['strong_attack', 'weak_attack',
            'dif_overall', 'dif_attack', 'dif_midfield', 'dif_deffense', 'dif_bu_speed',
            'dif_bu_passing','dif_cc_crossing', 'dif_cc_passing', 'dif_cc_shooting',
            'dif_aggression', 'dif_preassure', 'dif_avg_age']

X = FBall[features+countries]
X_win_train, X_win_test, y_win_train, y_win_test = tts(X,y_win,test_size=0.15)
X_draw_train, X_draw_test, y_draw_train, y_draw_test = tts(X,y_draw,test_size=0.15)
X_lose_train, X_lose_test, y_lose_train, y_lose_test = tts(X,y_lose,test_size=0.15)


win_classifier.fit(X=X_win_train,y=y_win_train)
draw_classifier.fit(X=X_draw_train,y=y_draw_train)
lose_classifier.fit(X=X_lose_train,y=y_lose_train)

[LibSVM][LibSVM][LibSVM]

SVC(C=1.0, cache_size=500, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=1e-08, verbose=3)

In [28]:
print(win_classifier.score(X=X_win_test,y=y_win_test),draw_classifier.score(X=X_draw_test,y=y_draw_test),lose_classifier.score(X=X_lose_test,y=y_lose_test))

0.5892857142857143 0.7321428571428571 0.8154761904761905


In [25]:
import matplotlib.pyplot as plt
%matplotlib inline

win_classifier.support_vectors_

array([[85., 79., 10., ...,  0.,  0.,  0.],
       [78., 77.,  0., ...,  0.,  0.,  0.],
       [87., 79.,  6., ...,  0.,  0.,  0.],
       ...,
       [76., 70.,  6., ...,  0.,  0.,  0.],
       [83., 77.,  3., ...,  0.,  0.,  0.],
       [79., 76.,  5., ...,  0.,  0.,  0.]])

In [ ]:
for i in range(len(win_classifier.feature_importances_)):
    print("{} : {}".format(features_new[i], win_classifier.feature_importances_[i]))

### Most important features:

dif_avg_age : 0.08617283329160567

weak_attack : 0.040044227024350405

dif_attack : 0.03344173497082109

dif_midfield : 0.04789722472672513

dif_bu_speed : 0.0498667226464478

dif_bu_passing : 0.0675497079085918

dif_cc_crossing : 0.05853012323549122

dif_cc_passing : 0.05038099088236613

dif_cc_shooting : 0.0477674420574618

dif_aggression : 0.04181026888097443

dif_preassure : 0.03826610674134294